In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ginipractice/Gini_and_Splitting_Test_Dataset


In [2]:
import numpy as np
import pandas as pd
from numpy import random 
from numpy import array

dataframe = pd.read_csv("../input/ginipractice/Gini_and_Splitting_Test_Dataset")

dataframe.columns = dataframe.iloc[0]
dataframe = dataframe.drop(0)
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.drop('Prediction', axis = 1)
dataframe.at[1, "X1"] = 7
dataframe.at[2, "X1"] = 8
dataframe.at[7, "X1"] = 2
dataframe.at[8, "X1"] = 3

for i in range(10):
    df = pd.DataFrame([[10*random.rand(), 10* random.rand(), random.randint(2)]],columns = dataframe.columns, index = [i+10])
    dataframe = dataframe.append(df)

dataset = dataframe.values.astype('float32')


In [3]:
dataframe

,X1,X2,Y
0,2.34387538,2.05175782,0
1,7,3.03293253,0
2,8,2.78632776,0
3,3.65634293,2.58146077,0
4,2.85319439,1.05233106,0
5,8.90764784,3.73054086,1
6,9.75246451,3.74075462,1
7,2,3.01340825,1
8,3,2.43633348,1
9,7.14252517,3.6501208,1


In [4]:
arr = array(dataset)
arr

array([[2.3438754 , 2.0517578 , 0.        ],
       [7.        , 3.0329325 , 0.        ],
       [8.        , 2.7863278 , 0.        ],
       [3.656343  , 2.5814607 , 0.        ],
       [2.8531945 , 1.0523311 , 0.        ],
       [8.907648  , 3.7305408 , 1.        ],
       [9.752464  , 3.7407546 , 1.        ],
       [2.        , 3.0134082 , 1.        ],
       [3.        , 2.4363334 , 1.        ],
       [7.142525  , 3.6501207 , 1.        ],
       [8.715614  , 9.322781  , 1.        ],
       [1.0397823 , 5.332135  , 1.        ],
       [2.199744  , 4.833983  , 0.        ],
       [7.1164336 , 0.28132018, 0.        ],
       [1.0972799 , 5.9210157 , 0.        ],
       [3.7463765 , 2.4884799 , 1.        ],
       [2.1081154 , 3.07392   , 0.        ],
       [5.7746754 , 6.999682  , 1.        ],
       [7.496687  , 1.5652717 , 0.        ],
       [3.193025  , 7.80576   , 0.        ]], dtype=float32)

In [5]:
from numpy import array
import numpy as np

def calc_best_gini(arr):
    min_scorer =  arr[0, 0]
    min_score = 1
    column = -1
    for row in arr:
        for col in range(1,3):
            tmp = row[col-1]
            group1, group2 = organize(tmp,col-1, arr)
            g10, g11, g20, g21 = 0, 0,0,0
            ng1, ng2 = len(group1), len(group2)
            for el in group1:
                if el[-1] == 0:
                    g10 += 1
                else:
                    g11 += 1
            for el in group2:
                if el[-1] == 0:
                    g20 += 1
                else:
                    g21 += 1
                
            if len(group1) == 0:
                g10, g11 = 0, 0
            else:
                g10, g11 = g10/len(group1), g11/len(group1)
            
            if len(group2) == 0:
                g20, g21 = 0, 0
            else:
                g20, g21 = g20/len(group2), g21/len(group2)
                    
            score = get_gini(10, ng1, ng2, g10, g11, g20, g21)
#             print("Column: X{}, Value: {}, Score: {}".format(col, tmp, score))
            if(min_score > score):
                min_score = score
                min_scorer = tmp
                column = col
    return min_scorer, column

def organize(tmp, col, arr):
    group1 = np.empty((0,3))
    group2 = np.empty((0,3))
    for row in arr:
        if(row[col] < tmp):
            group1 = np.append(group1, row.reshape((1, len(row))), 0)
        else:
            group2 = np.append(group2, row.reshape((1, len(row))), 0)
    return group1, group2

            
def get_gini(n, ng1, ng2, g10, g11, g20, g21):
    return ((1-(g10**2 + g11**2))*(ng1/n)) +((1-(g20**2 + g21**2))*(ng2/n)) 

In [6]:
min_scorer, column = calc_best_gini(arr)
group1, group2 = organize(min_scorer, column-1, arr)

min_scorer

8.715614

In [7]:
def recursive_split(arr):
    val = arr[0, 2]
    same = True
    for row in arr:
        if row[2] != val:
            same = False
            break
    if len(arr) <= 3 or same:
        ones, zeroes = 0,0
        for i in arr:
            if i[-1] == 1:
                ones+= 1
            else:
                zeroes+=1
        return 1 if max(zeroes, ones) == ones else 0 #returns the prediction from this branch of the tree based on the max-occurring value of binary class-counts
    
    comb_arr = []
    min_scorer, column = calc_best_gini(arr)
    group1, group2 = organize(min_scorer, column-1, arr)

#     comb_arr.append("Threshold:{} Category:{}".format(min_scorer, column))

    comb_arr.append((min_scorer, column))
    comb_arr.append(recursive_split(group1))
    comb_arr.append(recursive_split(group2))
    
    return comb_arr

def caller(arr):
    return recursive_split(arr)

In [8]:
x = caller(arr)

In [9]:
x

[(8.715614, 1),
 [(2.436333417892456, 2),
  0,
  [(2.58146071434021, 2),
   1,
   [(1.0972799062728882, 1),
    1,
    [(5.774675369262695, 1),
     [(2.1081154346466064, 1), 1, 0],
     [(3.650120735168457, 2), 0, 1]]]]],
 1]

In [10]:
import pprint

pp = pprint.PrettyPrinter(indent = 10, width= 100)
pp.pprint(x)

[         (8.715614, 1),
          [         (2.436333417892456, 2),
                    0,
                    [         (2.58146071434021, 2),
                              1,
                              [         (1.0972799062728882, 1),
                                        1,
                                        [         (5.774675369262695, 1),
                                                  [(2.1081154346466064, 1), 1, 0],
                                                  [(3.650120735168457, 2), 0, 1]]]]],
          1]


In [11]:
type(x) == list

True

In [12]:
def dfs(arr):
    
    dfs_arr = []
    
    for el in arr:
        if(type(el) == list):
            for el1 in dfs(el):
                dfs_arr.append(el1)
        else:
            dfs_arr.append(el)
    
    return dfs_arr

In [13]:
y = dfs(x)

In [14]:
class Node():
    def __init__(self, val, ind, prev = None, left = None, right = None):
        self.ind = ind
        self.val = val
        self.prev = prev
        self.left = left
        self.right = right

def construct(arr):
    if(type(arr[0]) != tuple):
        if len(arr) > 0:
            return arr[1:], Node(arr[0], -1)
        else:
            return [], Node(arr[0], -1)
    
    root = Node(arr[0][0], arr[0][1])
    rem, root.left = construct(arr[1:])
    rem, root.right = construct(rem)
    root.left.prev , root.right.prev = root, root
    
    return rem, root

In [15]:
def classify(tree, entry): #entry is a sample of data in the form [X1, X2] for which a prediction is outputted
    node = tree
    while node.left and node.right:
        cond = node.val
        ind = node.ind
        if(entry[ind-1] >= node.val):
            node = node.right
        else:
            node = node.left
    
    return node.val

In [16]:
y

[(8.715614, 1),
 (2.436333417892456, 2),
 0,
 (2.58146071434021, 2),
 1,
 (1.0972799062728882, 1),
 1,
 (5.774675369262695, 1),
 (2.1081154346466064, 1),
 1,
 0,
 (3.650120735168457, 2),
 0,
 1,
 1]

In [17]:
rem, tree = construct(y)

In [18]:
correct = 0

for entry in dataset:
    print("Entry: {}".format(entry),"Prediction: {}".format(classify(tree, entry[: -1])))
    if classify(tree, entry[: -1]) == entry[-1]: correct +=1
    else: 
        print("INCORRECT")
    print()

print("Accuracy: {}%".format(correct/len(dataset)*100))

Entry: [2.3438754 2.0517578 0.       ] Prediction: 0

Entry: [7.        3.0329325 0.       ] Prediction: 0

Entry: [8.        2.7863278 0.       ] Prediction: 0

Entry: [3.656343  2.5814607 0.       ] Prediction: 0

Entry: [2.8531945 1.0523311 0.       ] Prediction: 0

Entry: [8.907648  3.7305408 1.       ] Prediction: 1

Entry: [9.752464  3.7407546 1.       ] Prediction: 1

Entry: [2.        3.0134082 1.       ] Prediction: 1

Entry: [3.        2.4363334 1.       ] Prediction: 1

Entry: [7.142525  3.6501207 1.       ] Prediction: 1

Entry: [8.715614 9.322781 1.      ] Prediction: 1

Entry: [1.0397823 5.332135  1.       ] Prediction: 1

Entry: [2.199744 4.833983 0.      ] Prediction: 0

Entry: [7.1164336  0.28132018 0.        ] Prediction: 0

Entry: [1.0972799 5.9210157 0.       ] Prediction: 1
INCORRECT

Entry: [3.7463765 2.4884799 1.       ] Prediction: 1

Entry: [2.1081154 3.07392   0.       ] Prediction: 0

Entry: [5.7746754 6.999682  1.       ] Prediction: 1

Entry: [7.496687  1.5